The contents of this script are independently licensed under the Creative Commons Attribution 4.0 International License (CC BY 4.0). This licensing applies solely to this script and does not affect the licensing terms of the source repository, should they differ. © 2024 Luca Kunz.

TRAPs classes
==

This file contains a one function applied during the TRAPs postprocessing and TRAPs class applied in all pickle saving functions.

In [1]:
import os
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
import time
import datetime
import pickle
import geopandas
from shapely.geometry import Point, LineString, Polygon, asMultiPoint

# Interpolation along curve function

In [2]:
def interpol_along_curve(xs, ys, interpolation_mode):
    '''
    This function interpolates TRAP points to equal distances along a TRAP curve.
    '''
    
    # make sure that xs and ys are numpy arrays
    xs = np.array(xs)
    ys = np.array(ys)
    
    # get the coordinate distances to the previous TRAP point, is 0 for the first point on the curve
    xs_distances_to_previous = np.hstack((0,np.diff(xs))) # insert 0 at the array beginning
    ys_distances_to_previous = np.hstack((0,np.diff(ys))) # insert 0 at the array beginning
    # get the arclength distances to the previous point
    distances = np.sqrt(xs_distances_to_previous**2 + ys_distances_to_previous**2)
    
    # get the cummulative sum of the arclength distance from point to point along the TRAP curve. 
    # the last element of s will indicate the total length of the TRAP curve, the first element will be 0
    s = np.cumsum(distances)
    
    # assert that the last element is also the maximal value
    assert s[-1]==max(s), 'error generating cumsum array'

    # double check the total curve length
    assert np.round(s[-1], decimals=5)==np.round(np.sum(distances), decimals=5), 'error generating cumsum array'
    
    # save the total arclength of the curve
    total_curve_length = s[-1]
    
    # now map the original points on the curve parameterisation (in terms of arclength distance from the first point) 
    # to their values in the cartesian space and get an interpolation function for everything in between
    s_to_xs_interpolfunc = interp1d(s, xs, 'linear') # prefer linear interpolation since we calculated s linearly
    s_to_ys_interpolfunc = interp1d(s, ys, 'linear')
    
    # define to which arclength distance the equidistant TRAP points shall be interpolated
    interpol_distance = 1/12 # the velocity grid resolution is 1/4°, choose something finer, e.g. 1/12°
    # get the number of newly interpolated points
    num_interpol_points = round(total_curve_length/interpol_distance) + 1 # HERE WE INTRODUCE A ROUNDING ERROR

    # curve parameterisation positions of the interpolation points
    if interpolation_mode=='ARANGE':
        si = np.arange(0, total_curve_length, interpol_distance) # cuts off the last original TRAP point
        if total_curve_length==0: si = np.array([0.]) 
        # if there's only the first original TRAP point, np.arange(0,0,interpol_distance) would cause si=[] and 
        # throw the empty interpolation array assertion below, but we want this point and thus we prevent this case        
    elif interpolation_mode=='LINSPACE':
        si = np.linspace(0, total_curve_length, num_interpol_points) # keeps the last original TRAP point but distance varies

    # get the cartesian coordinates of the interpolated points along the curve
    # these points are equidistant since we hand over equidistant si-values to the interpolation function
    # which interpolates and maps them to cartesian coordinates
    xsi = s_to_xs_interpolfunc(si)
    ysi = s_to_ys_interpolfunc(si)

    # assert that the interpolation gave the right number of points and produced no NaN values
    assert xsi.size==ysi.size, 'unequal number of interpolation points in x and y'
    assert (np.all(~np.isnan(xsi)) and np.all(~np.isnan(ysi))), 'interpolation arrays bear NaN values'
    assert xsi.size <= num_interpol_points, 'more interpolation points than expected'
    
    # assert interpolated curves consist of at least one TRAP point which represents the first original coordinate
    assert xsi.size > 0, 'empty interpolation array, total curve length: ' + str(total_curve_length)
    assert xsi[0]==xs[0], 'first interpolated coordinate does not equal first original coordinate'
    assert ysi[0]==ys[0], 'first interpolated coordinate does not equal first original coordinate'
    
    return xsi, ysi
    

# Classes

In [3]:
class TRAPSdata:
    """
    This class objectifies the yearly full TRAPS DataFrame of a given velocity product.
    """ 
        
    def __init__(self, product_short, product_long, pd_TRAPS_df):
        
        self.product_short = product_short # the short name of the underlying velocity product
        self.product_long = product_long # the long name of the underlying velocity product
        self.pd_TRAPS_df = pd_TRAPS_df # the DataFrame containing all TRAP cores and curves
        